In [1]:
import pandas as pd

def dew_tem(tem,hum):
    #hum은 0~1 사이값
    import numpy as np
    a=18.678
    b=234.5
    c=257.14
    part1= a*b*c+tem**2-b*(c+tem)*np.log(hum)
    part2= 4*c*(c+tem)*((a*b-tem)*tem+b*(c+tem)*np.log(hum))
    total= (part1-(part1**2-part2)**0.5)/(2*(c+tem))
    return total

## Preprocessing
- asos 기상관측 자료와 시간 칼럼 추가

In [3]:
df=pd.read_csv("data.csv",encoding='utf-8')

In [4]:
asos=pd.read_csv("train_data/plant_asos.csv",encoding='utf-8')

In [6]:
df=df.merge(asos)

In [8]:
for i in df.columns:
    if df[i].isnull().sum()>50000:
        print(i)

In [14]:
df.interpolate(method='values',inplace=True)

In [17]:
column_01=['year', 'month', 'day', 'hour', 'week_1', 'week_2',
       'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'week_9',
       'week_10', 'week_11', 'week_12', 'week_13', 'week_14', 'week_15',
       'week_16', 'week_17', 'week_18', 'week_19', 'week_20', 'week_21',
       'week_22', 'week_23', 'week_24', 'week_25', 'week_26', 'week_27',
       'week_28', 'week_29', 'week_30', 'week_31', 'week_32', 'week_33',
       'week_34', 'week_35', 'week_36', 'week_37', 'week_38', 'week_39',
       'week_40', 'week_41', 'week_42', 'week_43', 'week_44', 'week_45',
       'week_46', 'week_47', 'week_48', 'week_49', 'week_50', 'week_51',
       'week_52','sky_1_24hr', 'sky_2_24hr', 'sky_3_24hr', 'sky_4_24hr',
       'rain_t_0_24hr', 'rain_t_1_24hr', 'rain_t_2_24hr', 'rain_t_3_24hr']
df.columns

Index(['mea_ddhr', 'cond_24hr', 'tem_in_loc_24hr', 'hum_in_loc_24hr',
       'tem_coil_loc_24hr', 'year', 'month', 'day', 'hour', 'week_1', 'week_2',
       'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'week_9',
       'week_10', 'week_11', 'week_12', 'week_13', 'week_14', 'week_15',
       'week_16', 'week_17', 'week_18', 'week_19', 'week_20', 'week_21',
       'week_22', 'week_23', 'week_24', 'week_25', 'week_26', 'week_27',
       'week_28', 'week_29', 'week_30', 'week_31', 'week_32', 'week_33',
       'week_34', 'week_35', 'week_36', 'week_37', 'week_38', 'week_39',
       'week_40', 'week_41', 'week_42', 'week_43', 'week_44', 'week_45',
       'week_46', 'week_47', 'week_48', 'week_49', 'week_50', 'week_51',
       'week_52', 'dj_aws_tem', 'dj_aws_wind_d', 'dj_aws_wind_s', 'dj_aws_hum',
       'sp_aws_tem', 'sp_aws_wind_d', 'sp_aws_wind_s', 'sp_aws_spot_pr',
       'sp_aws_sea_pr', 'tem_24hr', 'hum_24hr', 'rain_p_24hr', 'wind_s_24hr',
       'wind_d_24hr', 'sky_1_24

In [18]:
X=df.drop(['mea_ddhr', 'cond_24hr', 'tem_in_loc_24hr', 'hum_in_loc_24hr',
       'tem_coil_loc_24hr'],axis=1)

In [21]:
y=df.loc[:,'hum_in_loc_24hr']

## train_test_split
- 온도는 예측이 잘되는 반면 습도가 예측이 잘되지 않아 습도 위주로 예측

In [22]:
train_X=X.iloc[10000:]
test_X=X.iloc[:10000]
train_y=y.iloc[10000:]
test_y=y.iloc[:10000]

## xgboost 로 회귀

In [26]:
from xgboost import XGBRegressor
model=XGBRegressor()

In [27]:
model.fit(train_X,train_y)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[18:49:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [29]:
pre=model.predict(test_X)

## Modle_evaluation

In [32]:
m=pre.mean()
s=pre.std()
m1=y.mean()
s1=y.std()

new_pre=[]
for i in pre:
    new_pre.append(((i-m)/s)*s1+m1)

In [36]:
from sklearn.metrics import r2_score

In [39]:
r2_score(test_y,new_pre),r2_score(test_y,pre)

(0.438238739927881, 0.5104684820667762)

In [45]:
m_d=df.loc[:,['mea_ddhr', 'cond_24hr', 'tem_in_loc_24hr', 'hum_in_loc_24hr',
       'tem_coil_loc_24hr']].iloc[:10000]

In [46]:
t_d=pd.DataFrame({'test_y':test_y,'pre':pre,'new_pre':new_pre})

In [49]:
last_df=pd.concat([m_d,t_d],axis=1)

In [50]:
last_df

,mea_ddhr,cond_24hr,tem_in_loc_24hr,hum_in_loc_24hr,tem_coil_loc_24hr,test_y,pre,new_pre
0,2016-04-01 03:00:00,0,14.00,41.00,10.00,41.00,52.621311,51.419553
1,2016-04-01 06:00:00,0,13.00,38.00,11.00,38.00,52.724461,51.560172
2,2016-04-01 09:00:00,0,14.00,41.00,11.00,41.00,50.408161,48.402466
3,2016-04-01 12:00:00,0,17.00,35.00,13.00,35.00,50.109104,47.994775
4,2016-04-01 15:00:00,0,19.00,32.00,14.00,32.00,43.912563,39.547311
...,...,...,...,...,...,...,...,...
9995,2018-02-10 03:00:00,0,1.88,44.11,1.17,44.11,41.403507,36.126830
9996,2018-02-10 04:00:00,0,2.22,39.84,1.48,39.84,37.463192,30.755177
9997,2018-02-10 05:00:00,0,2.19,38.40,1.51,38.40,40.797913,35.301249
9998,2018-02-10 06:00:00,0,1.76,36.36,1.36,36.36,41.417679,36.146149


## 이슬점온도 계산후 evaluation

In [75]:
d_list=[]
d_tf=[]
for i in range(len(last_df)):
    d=dew_tem(last_df.tem_in_loc_24hr[i],last_df.new_pre[i]/100)+3
    d_list.append(d)
    d_tf.append(d>last_df.tem_coil_loc_24hr[i])

In [76]:
last_df['dew']=d_list
last_df['d_pred']=d_tf

In [77]:
from sklearn.metrics import f1_score
f1_score(d_tf,last_df.cond_24hr)

0.08391608391608392

In [78]:
pd.crosstab(last_df.d_pred,last_df.cond_24hr)

cond_24hr,0,1
d_pred,,
False,9863,28
True,103,6


In [79]:
last_df[last_df.cond_24hr==1]

,mea_ddhr,cond_24hr,tem_in_loc_24hr,hum_in_loc_24hr,tem_coil_loc_24hr,test_y,pre,new_pre,dew,d_pred
83,2016-04-11 12:00:00,1,18.00,78.00,14.00,78.00,47.153458,43.965476,8.564840,False
111,2016-04-15 00:00:00,1,17.00,80.00,14.00,80.00,49.710075,47.450797,8.754270,False
232,2016-04-30 03:00:00,1,21.00,80.00,18.00,80.00,56.765827,57.069584,15.182945,False
248,2016-05-02 03:00:00,1,21.00,80.00,18.00,80.00,70.955025,76.413074,19.695506,True
599,2016-06-15 00:00:00,1,24.00,78.00,19.00,78.00,60.630650,62.338321,19.360890,True
600,2016-06-15 03:00:00,1,23.00,76.00,19.00,76.00,66.186356,69.912163,20.217039,True
601,2016-06-15 06:00:00,1,23.00,74.00,19.00,74.00,62.693638,65.150699,19.107325,True
2069,2017-01-26 02:00:00,1,7.01,78.79,4.26,78.79,43.664879,39.209655,-2.928159,False
2547,2017-02-15 18:00:00,1,11.62,68.01,6.05,68.01,52.262959,50.931027,4.812981,False
2548,2017-02-15 19:00:00,1,11.62,72.99,6.09,72.99,52.558403,51.333793,4.923099,False
